In [ ]:
from datasets import load_dataset

dataset = load_dataset("amazon_polarity")
dataset

In [ ]:
dataset.set_format("pandas")

In [ ]:
from arize.pandas.embeddings import EmbeddingGenerator, UseCases
model_name = "arize-ai/distilbert_reviews_with_language_drift"
generator = EmbeddingGenerator.from_use_case(
    use_case=UseCases.NLP.SEQUENCE_CLASSIFICATION,
    model_name=model_name,
)


In [ ]:
train_df = dataset["train"][:1000]
test_df = dataset["test"][:100]

In [ ]:
train_df["content_vector"] = generator.generate_embeddings(text_col=train_df["content"])
test_df["content_vector"] = generator.generate_embeddings(text_col=test_df["content"])

In [ ]:
import phoenix as px


schema = px.Schema(
    actual_label_column_name="label",
    embedding_feature_column_names={
        "text_embedding": px.EmbeddingColumnNames(
            vector_column_name="content_vector", raw_data_column_name="content"
        ),
    },
)


prod_ds = px.Dataset(dataframe=test_df, schema=schema, name="testing")
train_ds = px.Dataset(dataframe=train_df, schema=schema, name="training")



In [ ]:
session = px.launch_app(primary=prod_ds, reference=train_ds)


In [ ]:
px.active_session().view()